
# River telemetry simulation

## Usage:
Modify the simulation parameters in the fields below, then run the simulation cell.

In [18]:
# @title Simulation cell
from dataclasses import dataclass
import random
from typing import List, Set
import numpy as np
from ipywidgets import interact, widgets
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import time

random.seed(None)

# Helper function(s)
def create_fish(fish_count: int, river_length: float):
  fish: List[float] = []
  for _ in range(fish_count):
    fish.append(river_length * random.random())

  return fish

# Simulation function
def run_simulation_cases(
    read_radius: float,
    read_time: float,
    fish_count: int,
    river_length: float,
    float_speed: float,
    cases_count: int
):
  fish_cases: List[List[float]] = []
  found_fish_cases: List[Set[int]] = []

  for _ in range(cases_count):
    fish_cases.append(create_fish(fish_count, river_length))
    found_fish_cases.append(set())

  current_frequency: int = 0
  frequency_switch_timer: float = 0

  print('Simulation progress')

  for t in tqdm(np.arange(0.0, river_length, float_speed)):
    if frequency_switch_timer >= read_time:
      if current_frequency == fish_count - 1:
        current_frequency = 0
      else:
        current_frequency += 1

      frequency_switch_timer = 0

    boat_position: float = t

    for i in range(cases_count):
      if abs(fish_cases[i][current_frequency] - boat_position) <= read_radius:
        found_fish_cases[i].add(current_frequency)

    frequency_switch_timer += 1

  results = np.array(list(map(
      lambda s: len(s),
      found_fish_cases
  )))
  return results

# Algebraic estimate
def algebraic_estimate(read_time: float, read_radius: float, float_speed: float):
  return (2 * read_radius) / (float_speed * read_time)

# Input
# (m)
read_radius = 100 # @param {"type":"number"}
# (s)
read_time = 6 # @param {"type":"number"}
fish_count = 30 # @param {"type":"number"}
# (m)
river_length = 10000 # @param {"type":"number"}
# (m/s)
float_speed = 2 # @param {"type":"number"}
test_cases = 100 # @param {"type":"number"}

dawn = time.time()
data = run_simulation_cases(read_radius, read_time, fish_count, river_length, float_speed, test_cases)
print(f'\nResult summary (fish found):\nMax: {data.max()}\nMin: {data.min()}\nMean: {data.mean()}\nVariance: {data.var()}')
dusk = time.time()
print(f'Simulation took {dusk - dawn} ms')

alg_est = algebraic_estimate(read_time, read_radius, float_speed)
print(f'\nAlgebraic estimate: {alg_est}\n')

Simulation progress


  0%|          | 0/500 [00:00<?, ?it/s]


Result summary (fish found):
Max: 23
Min: 10
Mean: 16.74
Variance: 7.5524000000000004
Simulation took 0.03762221336364746 ms

Algebraic estimate: 16.666666666666668

